In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader

import pandas as pd

import os
import re

from dotenv import load_dotenv
load_dotenv()

def get_file_names(folder_path, format=".pdf"):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.
    """
    import os
    
    try:
        all_files = os.listdir(folder_path)
        pdf_files = [file.replace(format,"") for file in all_files if file.lower().endswith(format)]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []
    

In [2]:
from loader import load_process_split_doc, load_process_split_doc_law

# tax_etc 파일들 정의의
doc_options = [
    {
        "filename":"연말정산_신고안내", 
        "page_ranges": [(17, 426)],
        "table_pages_range": "17-425",
        "replace_string": [
            {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"},
            {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"  },
            {"target_str": r"II. 2024년 귀속 연말정산 주요 일정\n\d+|"},
            {"target_str": r"III\. 원천징수의무자의 연말정산 중점 확인사항\n\d+|"},
            {"target_str": r"원천징수의무자를 위한 \n2024년 연말정산 신고안내\n\d+|"},
            {"target_str": r"Ⅰ\. 근로소득\n\d+|"},
            {"target_str": r"II\. 근로소득 원천징수 및 연말정산\n\d+|"},
            {"target_str": r"III\. 근로소득공제, 인적공제, 연금보험료공제\n\d+|"},
            {"target_str": r"IV\. 특별소득공제(소법 §52)\n\d+|"},
            {"target_str": r"V\. 그 밖의 소득공제(조특법)\n\d+|"},
            {"target_str": r"VI\. 세액감면(공제) 및 농어촌특별세\n\d+|"},
            {"target_str": r"I\. 2024년 귀속 연말정산 종합사례\n\d+|"},
            {"target_str": r"II\. 근로소득 원천징수영수증(지급명세서) 작성요령\n\d+|"},
            {"target_str": r"IV\. 수정 원천징수이행상황신고서 작성사례(과다공제)\n\d+|"},
            {"target_str": r"VI\. 홈택스를 이용한 연말정산 신고(근로소득 지급명세서 제출)\n\d+|"},
            {"target_str": r"I\. 사업소득 연말정산\n\d+|"},
            {"target_str": r"II\. 연금소득 연말정산\n\d+|"},
            {"target_str": r"I\. 종교인소득이란?\n\d+|"},
            {"target_str": r"IV\. 종교인소득(기타소득)에 대한 연말정산\n\d+|"},
            {"target_str": r"부록1\. 연말정산 관련 서비스\n\d+|"},
            {"target_str": r"부록2\. 연말정산간소화 서비스\n\d+|"},
            {"target_str": r"부록5\. 연말정산 주요 용어 설명\n\d+|"},
            {"target_str": r"부록6\. 소득·세액공제신고서 첨부서류\n\d+|"},
            {"target_str": r"\bNaN\b"},
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])","replace_str": r"\1\2"},
            {"target_str": r"\s+", "replace_str": r" "},
        ]
    },
    {
        "filename":"연말정산_주택자금·월세액_공제의이해", 
        "page_ranges": [(9, 12), (15, 24), (27, 28), (31, 39), (43, 70)],
        "table_pages_range": "9-12,15-24,27-28,31-39,43-70",
        "replace_string": [
            {"target_str": (r"연말정산 주택자금･월세액 공제의 이해\n\d+|" r"\bNaN\b"),  },
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
            {"target_str": r"\s+", "replace_str": " " },
        ]
    },
    {
        "filename":"2024_핵심_개정세법", 
        "page_ranges": [(19, 44), (47, 71), (75, 161)],
        "table_pages_range": "19-44,47-71,75-161",
        "replace_string": [
            {"target_str": (r"2\n0\n2\n5\n\s*달\n라\n지\n는\n\s*세\n금\n제\n도|"  
                r"\n2\n0\n2\n4\n\s*세\n목\n별\n\s*핵\n심\n\s*개\n정\n세\n법|"
                r"\n2\n0\n2\n4\n\s*개\n정\n세\n법\n\s*종\n전\n-\n개\n정\n사\n항\n\s*비\n교\n|"
                r"\s*3\s*❚국민･기업\s*납세자용\s*|"
                r"\s*2\s*0\s*2\s*4\s|"
                r"\s한국세무사회\s|" 
                r"\n7\n❚국민･기업 납세자용|"
                r"\n71\n❚상세본|"),  
            },
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
            {"target_str": r"\s+", "replace_str": " " },
        ]
    },
    {
        "filename":"주요_공제_항목별_계산사례", 
        "page_ranges": [(1,1), (4, 21)],
        "table_pages_range": "1,4-21",
        "replace_string": [
            {"target_str": r"\bNaN\b"},
            {"target_str": r"\s+", "replace_str": " " },
        ]
    }
    
]

# vector store 에 넣을 데이터 document 모으기 -> all_documents
all_documents = []
# load - 세법 
law_files = get_file_names("data/tax_law")
for file in law_files:
    all_documents.extend(load_process_split_doc_law(file))
# load - 참고자료
for doc in doc_options:
    all_documents.extend(load_process_split_doc(**doc))
    

c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 15 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 37 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 223 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 1 of document data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\par

In [7]:
len(all_documents)

6241

In [4]:
# 📌 vector store 생성
COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

def set_vector_store(documents):
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

    return Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )
    
vector_store = set_vector_store(all_documents)

In [18]:
COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

# 📌 vector store 불러오기
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = Chroma(
        embedding_function=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )



In [8]:
vector_store._collection.count()

6241

In [9]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 5, "lambda_mult":1}
)

In [28]:
# Prompt Template 생성
messages = [
        ("ai", """
        **기본 원칙**  
        1. 모든 답변은 학습된 세법 데이터 내에서만 작성하세요.  
        - 데이터에 없는 내용은 추측하거나 임의로 생성하지 마세요.  
        - 답변할 정보가 없을 경우, 정직하게 "잘 모르겠습니다."라고 응답하세요.  

        2. 질문을 정확히 이해하고 답변하세요.  
        - 조사(은, 는, 이, 가 등)를 포함한 문장 구조를 바르게 해석하세요.  
        - 질문의 맥락을 파악하고, 필요한 경우 후속 질문을 제안하세요.  

        **답변 작성 가이드**  
        3. 답변의 구성:  
        - **명료하고 간결**하게 작성하며, 비전문가도 이해할 수 있도록 용어를 풀어서 설명하세요.  
        - 질문과 관련된 법 조항, 시행령, 시행규칙의 **번호와 요약된 내용**을 포함하세요.  
        - 법, 시행령, 시행규칙의 계층 관계를 명확히 하세요:  
        - 법률: 과세 원칙 및 제도를 규정.  
        - 시행령: 대통령령으로 법률의 구체적인 사항을 규정.  
        - 시행규칙: 부처에서 정한 세부 절차와 요건.  
        - 필요 시 관련 조항의 전문을 인용하세요.  

        4. 연관 정보 활용:  
        - 질문이 포함된 조항뿐 아니라, 연관된 조항도 참고해 답변의 정확성과 완성도를 높이세요.  
        - 질문이 세부 적용 방법에 관한 것이라면, 시행규칙을 우선 참고하되 법률 및 시행령과의 연관성을 설명하세요.  


        **추가 규칙**  
        5. **법률, 시행령, 시행규칙 구분 응답**  
        - 사용자가 특정 법 조항을 요청할 경우, **법률, 시행령, 시행규칙**을 명확히 구분하여 답변하세요.  
        - 예: "개별소비세법 제1조"를 물어보면 개별소비세법.pdf 파일을 가장 우선으로 참고해서 대해 답변합니다.  
        - 예: "개별소비세법 시행령 제1조"를 물어보면 개별소비세법_시행령.pdf 파일을 가장 우선으로 참고해서 대해 답변합니다.  
        - 예: "개별소비세법 시행규칙 제1조"를 물어보면 개별소비세법_시행규칙.pdf 파일을 가장 우선으로 참고해서 대해 답변합니다.  

        **주의사항**  
        6. 답변 형식:  
        - 모든 답변은 법적 조언이 아닌 **정보 제공 목적**임을 명확히 하세요.  
        - 질문에 해당하지 않는 계층(법, 시행령, 시행규칙)의 정보는 포함하지 마세요.  

        7. 사용자 요청에 따라 세법 구조나 맥락을 간단히 설명해, 이해를 도우세요.

	{context}")"""
        ),
        ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain1 = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser

In [33]:
print(chain1.invoke("법인세법 제1조 에 대해 알려줘"))
retriever.invoke("법인세법 제1조 에 대해 알려줘")

법인세법 제1조는 법인세법의 목적에 대해 규정하고 있습니다. 일반적으로 법 제1조는 해당 법률의 목적과 기본적인 방향성을 설명합니다. 법인세법 제1조는 법인세의 부과 및 징수에 관한 사항을 규정함으로써 국가 재정에 기여하고, 법인의 경제 활동을 적절히 통제하기 위한 기본적인 목적을 명시하고 있을 것입니다.

자세한 조문 내용은 법인세법 원문을 참고하시기 바랍니다. 법률 문서에서 제1조는 대개 법의 목적을 설명하는 조항입니다. 추가적인 세부 사항이나 특정 질문이 있으시면 말씀해 주세요.


[Document(metadata={'description': '법인세법 시행규칙 에 관한 문서입니다.', 'keyword': '법인세법_시행규칙', 'source': '법인세법_시행규칙.pdf'}, page_content='법인세법 시행규칙\n \n기획재정부 (국제조세제도과(외국법인)) 044-215-4652\n기획재정부 (법인세제과) 044-215-4221\n       제1장 총칙'),
 Document(metadata={'description': '법인세법 시행령 에 관한 문서입니다.', 'keyword': '법인세법_시행령', 'source': '법인세법_시행령.pdf'}, page_content='법인세법 시행령\n \n기획재정부 (국제조세제도과(외국법인)) 044-215-4652\n기획재정부 (법인세제과) 044-215-4221\n       제1장 총칙'),
 Document(metadata={'description': '법인세법 에 관한 문서입니다.', 'keyword': '법인세법', 'source': '법인세법.pdf'}, page_content='법인세법\n \n기획재정부 (법인세제과) 044-215-4221\n기획재정부 (국제조세제도과(외국법인)) 044-215-4652\n       제1장 총칙')]

In [34]:
print(chain1.invoke("개별소비세법 제1조(과세대상과 세율) 에 대해 알려줘"))
retriever.invoke("개별소비세법 제1조(과세대상과 세율) 에 대해 알려줘")

개별소비세법 제1조는 과세대상과 세율에 관한 내용을 규정하고 있습니다. 이 조항은 개별소비세가 부과되는 대상과 그에 대한 세율을 명확히 하고 있습니다. 주요 내용은 다음과 같습니다:

1. **과세대상**:  
   - 특정한 물품
   - 특정한 장소에서의 입장행위
   - 특정한 장소에서의 유흥음식행위
   - 특정한 장소에서의 영업행위

2. **세율**: 
   - 물품에 대한 세율은 물품의 종류에 따라 다르게 적용됩니다. 예를 들어, 보석, 귀금속 제품, 고급 시계 등은 과세가격에 20%의 세율이 적용됩니다.
   - 입장행위에 대해선 장소에 따라 고정 금액으로 부과되며, 예를 들어 경마장의 경우 1명 1회 입장에 대해 1,000원이 부과됩니다.
   - 유흥음식행위 및 특정 영업장소에 대해서는 해당 요금의 일정 비율로 세율이 적용됩니다.

3. **특별 규정**:  
   - 세율은 경기 조절, 가격 안정 등의 필요에 따라 조정될 수 있습니다. 
   - 과세물품의 판정은 물품의 형태, 용도, 성질 등에 따라 이루어지며, 동일한 물품이 여러 품목에 해당할 경우 높은 세율이 적용되는 물품으로 취급됩니다.

이 조항은 개별소비세가 적용되는 구체적인 항목과 그에 따른 세율을 규정하여 세금 부과의 기준을 명확히 하고 있습니다.


[Document(metadata={'description': '개별소비세법 제1조 (과세대상과 세율)에 관한 문서입니다.', 'keyword': '개별소비세법', 'source': '개별소비세법.pdf'}, page_content='제1조(과세대상과 세율)  ① 개별소비세는 특정한 물품, 특정한 장소 입장행위(入場行爲), 특정한 장소에서의 유흥음식행\n위(遊興飮食行爲) 및 특정한 장소에서의 영업행위에 대하여 부과한다. \n② 개별소비세를 부과할 물품(이하 “과세물품”이라 한다)과 그 세율은 다음과 같다.\n1. 다음 각 목의 물품에 대해서는 그 물품가격에 100분의 20의 세율을 적용한다.\n가. 투전기(投錢機), 오락용 사행기구(射倖器具), 그 밖의 오락용품\n나. 수렵용 총포류\n2. 다음 각 목의 물품에 대해서는 그 물품가격 중 대통령령으로 정하는 기준가격(이하 “기준가격”이라 한다)을 초과\n하는 부분의 가격(이하 이 호에서 “과세가격”이라 한다)에 해당 세율을 적용한다.\n가. 다음의 물품에 대해서는 과세가격의 100분의 20\n1) 보석, 진주, 별갑(鼈甲), 산호, 호\n박(琥珀) 및 상아와 이를 사용한 제품(나석을 사용한 제품은 포함한다)\n2) 귀금속 제품\n3) 삭제\n4) 고급 시계\n5) 고급 융단\n6) 고급 가방\n나. 다음의 물품에 대해서는 과세가격의 100분의 20\n1) 고급 모피와 그 제품\n2) 고급 가구\n3. 다음 각 목의 자동차에 대해서는 그 물품가격에 해당 세율을 적용한다.\n가. 배기량이 2천시시를 초과하는 승용자동차와 캠핑용자동차: 100분의 5\n나. 배기량이 2천시시 이하인 승용자동차(배기량이 1천시시 이하인 것으로서 대통령령으로 정하는 규격의 것은\n제외한다)와 이륜자동차 : 100분의 5\n다. 전기승용자동차(「자동차관리법」 제3조제2항에 따른 세부기준을 고려하여 대통령령으로 정하는 규격의 것은\n제외한다): 100분의 5\n4. 다음 각 목의 물품에 대해서는 그 수량에 해당 세율을 적용한다.\n가. 휘발유

In [35]:
print(chain1.invoke("개별소비세법 과세대상과 세율 에 대해 알려줘"))
retriever.invoke("개별소비세법 과세대상과 세율 에 대해 알려줘")

개별소비세법의 과세대상과 세율에 대해 설명드리겠습니다.

### 개별소비세법 과세대상
개별소비세법에 따르면, 과세대상은 특정한 물품 및 용역으로, 주로 사치성 소비재 및 특정 서비스가 포함됩니다. 일반적으로 다음과 같은 항목들이 과세대상에 포함됩니다:
1. 특정 고가의 물품 (예: 고급 자동차, 고가의 가구)
2. 특정한 서비스 (예: 카지노, 골프장 이용)
3. 특정한 환경에 영향을 미치는 제품 (예: 에너지 소비가 높은 제품)

### 개별소비세법 세율
세율은 과세대상에 따라 다르게 적용됩니다. 일반적으로 다음과 같은 기준이 있습니다:
- **고급 자동차**: 일정 비율의 세율
- **사치품**: 일정 비율의 세율
- **환경에 영향을 미치는 제품**: 제품의 종류와 환경 영향에 따라 차등 적용

세율은 법률 및 시행령에 따라 구체적으로 명시되어 있으며, 각 물품 및 용역에 대해 다르게 설정될 수 있습니다.

보다 구체적인 세율 정보는 개별소비세법 및 시행령의 관련 조항을 참고하시기 바랍니다. 필요하시면 관련 조항 번호를 제공해드릴 수 있습니다.


[Document(metadata={'description': '개별소비세법 시행규칙 에 관한 문서입니다.', 'keyword': '개별소비세법_시행규칙', 'source': '개별소비세법_시행규칙.pdf'}, page_content='개별소비세법 시행규칙\n \n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336'),
 Document(metadata={'description': '개별소비세법 시행령 에 관한 문서입니다.', 'keyword': '개별소비세법_시행령', 'source': '개별소비세법_시행령.pdf'}, page_content='개별소비세법 시행령\n \n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336'),
 Document(metadata={'description': '개별소비세법 에 관한 문서입니다.', 'keyword': '개별소비세법', 'source': '개별소비세법.pdf'}, page_content='개별소비세법\n \n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336')]

In [37]:
print(chain1.invoke("개별소비세법 제1조 에 대해 알려줘"))
retriever.invoke("개별소비세법 제1조 에 대해 알려줘")

개별소비세법 제1조는 개별소비세법의 목적과 과세 대상에 관한 기본적인 사항을 규정하고 있습니다. 이 조항은 개별소비세가 부과되는 물품, 장소, 유흥장소 등을 명시하며, 이러한 것들이 과세 대상임을 설명합니다. 

예를 들어, 개별소비세법 시행령 제1조에서는 개별소비세법 제1조에 따른 과세물품의 세목, 과세장소의 종류, 과세유흥장소의 종류 등을 세부적으로 규정하고 있습니다. 

개별소비세법 제1조는 개별소비세의 적용 범위를 명시하는 중요한 조항이므로, 이 조항을 이해하는 것은 세법 전반을 이해하는 데 있어 중요합니다.

만약 더 구체적인 내용이나 세부 사항이 필요하다면, 관련 시행령이나 시행규칙을 참고하여 보다 자세한 정보를 확인하는 것이 좋습니다.


[Document(metadata={'description': '개별소비세법 시행규칙 에 관한 문서입니다.', 'keyword': '개별소비세법_시행규칙', 'source': '개별소비세법_시행규칙.pdf'}, page_content='개별소비세법 시행규칙\n \n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336'),
 Document(metadata={'description': '개별소비세법 시행령 에 관한 문서입니다.', 'keyword': '개별소비세법_시행령', 'source': '개별소비세법_시행령.pdf'}, page_content='개별소비세법 시행령\n \n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336'),
 Document(metadata={'description': '개별소비세법 시행령 제1조 (과세물품ㆍ과세장소 및 과세유흥장소의 세목등)에 관한 문서입니다.', 'keyword': '개별소비세법_시행령', 'source': '개별소비세법_시행령.pdf'}, page_content='제1조(과세물품ㆍ과세장소 및 과세유흥장소의 세목등)  「개별소비세법」 제1조제6항에 따른 과세물품의 세목은 별표\n1과 같이 하고, 과세장소의 종류는 별표 2와 같이 하며, 과세유흥장소의 종류는 유흥주점ㆍ외국인전용 유흥음식점\n및 그 밖에 이와 유사한 장소로 하고, 과세영업장소의 종류는 「관광진흥법」 제5조제1항에 따라 허가를 받은 카지노\n(「폐광지역개발 지원에 관한 특별법」 제11조에 따라 허가를 받은 카지노를 포함한다)로 한다.')]